<a href="https://colab.research.google.com/github/wooihaw/flowers_classification/blob/main/eight_flowers_cnn_transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Classification of eight types of flowers with transfer learning
##In this example, we are going to train a CNN model with transfer learning for the classfication of eight types of flowers

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Extract dataset from Google Drive
!cd /content/
!unzip /content/drive/MyDrive/eight_flowers.zip > /dev/null

In [ ]:
# Load modules
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

# Setup foldres for training, validation and testing images
train_dir = 'eight_flowers/train/'
test_dir = 'eight_flowers/test/'
val_dir = 'eight_flowers/valid/'

# Stream images from folders
train_datagen = ImageDataGenerator(rescale=1/255.)

test_datagen = ImageDataGenerator(rescale=1/255.)

val_datagen = ImageDataGenerator(rescale=1/255.)

train_generator = train_datagen.flow_from_directory(
                    train_dir,
                    batch_size=32,
                    target_size=(150,150),
                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
                    test_dir,
                    batch_size=32,
                    target_size=(150,150),
                    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
                    val_dir,
                    batch_size=32,
                    target_size=(150,150),
                    class_mode='categorical')

In [ ]:
# Import the inception mode, do not include the fully-connected layer at the top as the last layer of the network
base_model = InceptionV3(input_shape=(150, 150, 3), include_top=False, weights='imagenet')

# Make all the layers in the pre-trained model non-trainable
for layer in base_model.layers:
    layer.trainable = False

last_layer = base_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

# Applies average pooling on the spatial dimensions until each spatial dimension is one
x = GlobalAveragePooling2D()(last_output)
# Add a fully connected layer with 1024 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.2
x = Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = Dense(8, activation='softmax')(x)

model = Model(base_model.input, x)

model.compile(optimizer ='rmsprop',
              loss = 'categorical_crossentropy',
              metrics = ['acc'])

history = model.fit(train_generator,
                    steps_per_epoch=195,
                    epochs=3,
                    validation_data=val_generator,
                    validation_steps=19,
                    verbose=1)

In [ ]:
scores = model.evaluate(test_generator)
print(f"{model.metrics_names[0]}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]}: {scores[1]:.2%}")

In [ ]:
# Plot model accuracy and lose
acc = history.history['acc']
loss = history.history['loss']
val_acc = history.history['val_acc']
val_loss = history.history['val_loss']

plt.plot(np.arange(1, len(acc) + 1), acc, 'b', label='accuracy')
plt.plot(np.arange(1, len(val_acc) + 1), val_acc, 'g', label='val_accuracy')
plt.xlabel('Epoch')
plt.xticks(np.arange(1, len(acc) + 1))
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend(loc=0)
plt.show()

plt.plot(np.arange(1, len(loss) + 1), loss, 'b', label='loss')
plt.plot(np.arange(1, len(val_loss) + 1), val_loss, 'g', label='val_loss')
plt.xlabel('Epoch')
plt.xticks(np.arange(1, len(loss) + 1))
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend(loc=0)
plt.show()

In [ ]:
# load class indices
class_indices = train_generator.class_indices
map2class = {class_indices[k]:k for k in class_indices}
print(map2class)

In [ ]:
# Display the test image and show the predicted class
file = 'eight_flowers/test/sunflower/sunflower_test_0001.jpg'
img = load_img(file, target_size=(150, 150))
x = img_to_array(img)/255.
x = np.expand_dims(x, axis=0)
classes = model.predict(x)
print(classes)
for i, k in enumerate(class_indices):
  print(f'{k:10}: {classes[0, i]: .3f}')

plt.axis(False)
plt.imshow(load_img(file))
plt.title(f'Predicted as {map2class[classes.argmax()]}', y=0, pad=-10, verticalalignment="top")